In [1]:
import io
import time
import urllib

import numpy as np
import pandas as pd
import requests
from config import Config
from requests_ntlm import HttpNtlmAuth

In [2]:
class ContractMonitor:
    def __init__(self, hotel_id, domain, season):
        self.ssrs_url = (
            Config.SSRS_BASE_URL + domain + " Reports/Contracting/Contract Monitor"
        )
        self.ssrs_usr = Config.SSRS_USERNAME
        self.ssrs_pwd = Config.SSRS_PASSWORD
        self.payload = [
            ("Seas:isnull", True),
            ("Season", season),
            ("City", "ALL"),
            ("Hotel_Chain", "ALL"),
            ("hotel", hotel_id),
            ("operator", 1000000),
            ("CreateDateFrom:isnull", True),
            ("CreateDateTo:isnull", True),
            ("contracttype", "A"),
            ("contracttype", "S"),
            ("contractstatus", "E"),
            ("contractstatus", "H"),
            ("Revize", 2),
            ("showauth", True),
            ("view", 1),
            ("citycontract", 1),
            ("citycontract", 0),
            ("rs:ParameterLanguage", ""),
            ("rs:Command", "Render"),
            ("rs:Format", "CSV"),
            ("rc:ItemPath", "table1"),
        ]

        self.params = urllib.parse.urlencode(self.payload, quote_via=urllib.parse.quote)

    def get(self):
        response = requests.get(
            self.ssrs_url,
            params=self.params,
            stream=True,
            auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
        )
        print(response.url, response.status_code)
        if response.status_code == 200:
            data = response.content.decode("utf8")
            return data
        return None

In [3]:
contract_monitor = ContractMonitor(157243, "DU", "ALL 22-23")

In [4]:
data = contract_monitor.get()

http://95.211.42.206/ReportServer_SSRS?/DU%20Reports/Contracting/Contract%20Monitor&Seas%3Aisnull=True&Season=ALL%2022-23&City=ALL&Hotel_Chain=ALL&hotel=157243&operator=1000000&CreateDateFrom%3Aisnull=True&CreateDateTo%3Aisnull=True&contracttype=A&contracttype=S&contractstatus=E&contractstatus=H&Revize=2&showauth=True&view=1&citycontract=1&citycontract=0&rs%3AParameterLanguage=&rs%3ACommand=Render&rs%3AFormat=CSV&rc%3AItemPath=table1 200


In [5]:
pd.read_csv(io.StringIO(data)).to_csv("out.csv", index=False)

In [ ]:
df = pd.read_csv(
    io.StringIO(data),
    header=0,
    skiprows=2,
    names=[
        "contract_id",
        "contract_type",
        "status",
        "purchase_contract_id",
        "revise_number",
        "prev_contract_id",
        "is_revised",
        "hotel_id",
        "main_season",
        "create_date",
        "begin_date",
        "end_date",
        "res_start_date",
        "res_end_date",
        "meal_plan",
    ],
    usecols=[0, 1, 2, 3, 4, 5, 6, 8, 20, 21, 22, 23, 24, 25, 26],
    dtype={
        "contract_id": pd.Int64Dtype(),
        "contract_type": str,
        "status": str,
        "purchase_contract_id": pd.Int64Dtype(),
        "revise_number": pd.Int64Dtype(),
        "prev_contract_id": pd.Int64Dtype(),
        "is_revised": bool,
        "hotel_id": pd.Int64Dtype(),
        "main_season": str,
        "meal_plan": str,
    },
    parse_dates=[
        "create_date",
        "begin_date",
        "end_date",
        "res_start_date",
        "res_end_date",
    ],
)

In [ ]:
df.replace({pd.NaT: None, pd.NA: None, np.NaN: None}, inplace=True)

In [ ]:
from pretty_html_table import build_table

html_table_blue_light = build_table(
    df,
    "blue_light",
    font_size="13px",
    font_family="Arial, sans-serif",
    width="100px",
    text_align="center",
    padding="5px",
)

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from smtplib import SMTP


def send_mail(body):
    message = MIMEMultipart()
    message["Subject"] = "Test"
    message["From"] = Config.MAIL_USERNAME
    message["To"] = "ivan.orara@meetingpointuae.com"

    body_content = body
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()

    server = SMTP(Config.MAIL_SERVER, Config.MAIL_PORT)
    server.starttls()
    server.login(message["From"], Config.MAIL_PASSWORD)
    server.sendmail(message["From"], message["To"], msg_body)
    server.quit()

In [ ]:
send_mail(html_table_blue_light)

In [6]:
from config import Config

In [8]:
Config.SSRS_BASE_URL

'http://95.211.42.206/ReportServer_SSRS?/'

In [9]:
Config.SSRS_USERNAME

'RS_AMS01_FTIAS_VIEW'

In [10]:
Config.SSRS_PASSWORD


'Ft!r3p0rt$AMS'